# WavCeption V1: just a 1-D Inception approach 

저는 단지 제가 가지고 놀던 작은 장난감을 공유하고 싶었을 뿐인데 **놀라운 결과**를 주었습니다. 지금은 시간이 없기 때문에, 사람들이 어떻게 가지고 놀지 보려고 공유하려고 합니다. :-D. **WavCception V1** 네트워크는 일반 컨볼루션 뉴럴 네트워크에 비해 인상적인 결과를 얻을 수 있는 것으로 보이지만, 이번 경쟁에서는 전처리 및 알려지지 않은 트랙 관리에 많은 노력이 필요한 것으로 보입니다. 이것은 구글의 초기 네트워크를 기반으로 합니다. 같은 아이디어입니다. 

 저는 몇 주 전에 이러한 모듈들을 캐스케이드로 연결하여 쉽게 1D 인셉션 네트워크를 구축할 수 있도록 구현하는 모듈을 작성했습니다(아래 참조).
 
 아쉽게도 몇 가지 Kaggle의 제약으로 인해 커널 머신에서는 실행되지 않으므로 다운로드하여 자신의 머신에서 실행해 보시기 바랍니다.
 
 너무 고생하지 않고 12시간 동안 모델을 실행함으로써 리더보드에서 0.76을 달성했습니다(로컬 테스트에서는 0.84). 같은 라인의 다른 시험에서는 로컬에서 0.89를 얻었기 때문에 알려지지 않은 클립을 처리하는 방법이 크게 개선되었습니다.

## Load modules and libraries

In [10]:
%matplotlib inline
import numpy as np 
import pandas as pd 
import os
import shutil # shutil 모듈은 파일과 파일 모음에 대한 여러 가지 고수준 연산을 제공합니다. 특히, 파일 복사와 삭제를 지원하는 함수가 제공됩니다
import glob
import random

from tqdm import tqdm 
from collections import Counter # Counter('hello world') # Counter({'l': 3, 'o': 2, 'h': 1, 'e': 1, ' ': 1, 'w': 1, 'r': 1, 'd': 1})
from sklearn.preprocessing import LabelEncoder
import IPython
from numpy.fft import rfft, irfft # 1차원 이산 푸리에변환 # 역푸리에변환
import itertools

from scipy.io import wavfile
import IPython.display as ipd
import matplotlib.pyplot as plt
import scipy as sp
import tensorflow as tf

## Noise generation functions 

이 섹션의 코드는 다음 항목에서 차용 및 개조되었습니다.
https://github.com/python-acoustics/python-acoustics/blob/master/acoustics/generator.py

In [13]:
def ms(x):
    """Mean value of signal `x` squared.
    :param x: Dynamic quantity.
    :returns: Mean squared of `x`.
    """
    return (np.abs(x)**2.0).mean()

def normalize(y, x=None):
    """normalize power in y to a (standard normal) white noise signal.
    Optionally normalize to power in signal `x`.
    #The mean power of a Gaussian with :math:`\\mu=0` and :math:`\\sigma=1` is 1.
    """
    #return y * np.sqrt( (np.abs(x)**2.0).mean() / (np.abs(y)**2.0).mean() )
    if x is not None:
        x = ms(x)
    else:
        x = 1.0
    return y * np.sqrt( x / ms(y) )
    #return y * np.sqrt( 1.0 / (np.abs(y)**2.0).mean() )

def white_noise(N, state=None):
    state = np.random.RandomState() if state is None else state
    return state.randn(N)

def pink_noise(N, state=None):
    state = np.random.RandomState() if state is None else state
    uneven = N%2
    X = state.randn(N//2+1+uneven) + 1j * state.randn(N//2+1+uneven)
    S = np.sqrt(np.arange(len(X))+1.) # +1 to avoid divide by zero
    y = (irfft(X/S)).real
    if uneven:
        y = y[:-1]
    return normalize(y)

def blue_noise(N, state=None):
    """
    Blue noise. 
    
    :param N: Amount of samples.
    :param state: State of PRNG.
    :type state: :class:`np.random.RandomState`
    
    Power increases with 6 dB per octave.
    Power density increases with 3 dB per octave. 
    
    """
    state = np.random.RandomState() if state is None else state
    uneven = N%2
    X = state.randn(N//2+1+uneven) + 1j * state.randn(N//2+1+uneven)
    S = np.sqrt(np.arange(len(X)))# Filter
    y = (irfft(X*S)).real
    if uneven:
        y = y[:-1]
    return normalize(y)

def brown_noise(N, state=None):
    """
    Violet noise.
    
    :param N: Amount of samples.
    :param state: State of PRNG.
    :type state: :class:`np.random.RandomState`
    
    Power decreases with -3 dB per octave.
    Power density decreases with 6 dB per octave. 
    """
    state = np.random.RandomState() if state is None else state
    uneven = N%2
    X = state.randn(N//2+1+uneven) + 1j * state.randn(N//2+1+uneven)
    S = (np.arange(len(X))+1)# Filter
    y = (irfft(X/S)).real
    if uneven:
        y = y[:-1]
    return normalize(y)

def violet_noise(N, state=None):
    """
    Violet noise. Power increases with 6 dB per octave. 
    
    :param N: Amount of samples.
    :param state: State of PRNG.
    :type state: :class:`np.random.RandomState`
    
    Power increases with +9 dB per octave.
    Power density increases with +6 dB per octave. 
    
    """
    state = np.random.RandomState() if state is None else state
    uneven = N%2
    X = state.randn(N//2+1+uneven) + 1j * state.randn(N//2+1+uneven)
    S = (np.arange(len(X)))# Filter
    y = (irfft(X*S)).real
    if uneven:
        y = y[:-1]
    return normalize(y)

In [3]:
import numpy as np

N = 10
state = np.random.RandomState()
state.randn(N)

array([-0.59026399, -0.68431962,  1.56870611, -1.10500642, -0.67393401,
        0.10376222, -0.31171382, -0.35720405, -0.74424293,  0.12614498])

In [7]:
N//2+1+uneven

6

In [6]:
X = state.randn(N//2+1+uneven)
X

array([-0.17811637,  0.56738609,  0.1139559 , -0.1609064 , -0.0583465 ,
        0.09898859])

In [5]:
state = np.random.RandomState()
uneven = N%2 # 나머지
X = state.randn(N//2+1+uneven) + 1j * state.randn(N//2+1+uneven) #
X

array([-0.67854019+0.33696715j, -0.53639523-1.33228336j,
       -0.16807449-1.02284005j,  0.57717245+0.56470304j,
       -0.563283  +1.19573055j, -1.07630368+0.5818659j ])

In [8]:
S = np.sqrt(np.arange(len(X))+1.) # +1 to avoid divide by zero
S

array([1.        , 1.41421356, 1.73205081, 2.        , 2.23606798,
       2.44948974])

In [14]:
X = state.randn(N//2+1+uneven) + 1j * state.randn(N//2+1+uneven)
S = np.sqrt(np.arange(len(X))+1.) # +1 to avoid divide by zero
y = (irfft(X/S)).real

if uneven:
    y = y[:-1]
normalize(y)

array([-1.13220548,  0.52472321,  1.24908875,  0.27902739, -0.36896684,
       -0.32109447, -1.36113839, -1.96029794, -0.84810501, -0.38821552])

## Tensorflow utilities

텐서플로우 공통 작업을 모듈화하기 위한 유틸리티입니다.


In [ ]:
# Tf Utils
def get_tensorflow_configuration(device="0", memory_fraction=1): # memory_fraction=1 : 100% 사용 # configuration : 구성
    """
    사용할 GPU 및 프로세스에서 사용할 수 있는 메모리 양을 선택하는 기능입니다.
    1. device: 사용할 장치를 지정합니다(str)
    2. memory_fraction: 할당해야 하는 메모리의 비율입니다(float)
    3. return: 세션에 전달할 구성입니다(tf 개체)
    """
    device = str(device)
    config = tf.ConfigProto() 
    config.allow_soft_placement = True # 선택한 장치가 없으면 다른 장치로 대체
    config.gpu_options.per_process_gpu_memory_fraction = memory_fraction
    config.gpu_options.visible_device_list = device
    return(config)


def start_tensorflow_session(device="0", memory_fraction=1):
    """
    사용할 GPU 장치를 처리하는 텐서플로우 세션을 시작합니다.
    이 부분은 미리 인식될 메모리의 비율입니다.
    
    1. return: configured tf.Session
    """
    return(tf.Session(config=get_tensorflow_configuration(device=device, memory_fraction=memory_fraction)))


def get_summary_writer(session, logs_path, project_id, version_id):
    """
    For Tensorboard reporting
    1. session: opened tensorflow session (tf.Session)
    2. logs_path: 텐서보드가 로그를 찾는 경로입니다. (str)
    3. project_id: 보고용 프로젝트 이름입니다. (str)
    4. version_id: 보고용 버전 이름입니다.(str)
    5. return summary_writer: the tensorboard writer
    """
    path = os.path.join(logs_path,"{}_{}".format(project_id, version_id)) 
    
    if os.path.exists(path):
        shutil.rmtree(path) # 전체 디렉터리 트리를 삭제합니다
    
    summary_writer = tf.summary.FileWriter(path, graph_def=session.graph_def)
    
    return(summary_writer)


## Paths management module

경로를 처리할 모듈입니다.


In [ ]:
def abc(a):
    def ab(*arg, **kwargs)

In [ ]:
# Common paths
def _norm_path(path):
    """
    경로 검색 함수의 출력을 정규화하는 데 사용하기 위한 데코레이터 함수입니다.
    슬래시/백슬래시 windows 케이스를 수정하는 데 유용합니다.
    """
    def normalize_path(*args, **kwargs):
        return os.path.normpath(path(*args, **kwargs))
    
    return normalize_path


def _assure_path_exists(path): # assure : 확신
    """
    경로 검색 함수의 출력 여부를 확인하기 위한 데코레이터 함수입니다.
    fixing the slash/backslash windows cases.
    """
    def assure_exists(*args, **kwargs):
        p=path(*args, **kwargs)
        
        assert os.path.exists(p), "the following path does not exist: '{}'".format(p)
        return p
    
    return assure_exists


def _is_output_path(path):
    """
    출력 경로 검색 함수의 출력에 적용되는 함수를 그룹화하기 위한 데코레이터 함수입니다.
    """
    @_norm_path
    @_assure_path_exists
    def check_existence_or_create_it(*args, **kwargs):
        if not os.path.exists(path(*args, **kwargs)):
            "Path does not exist... creating it: {}".format(path(*args, **kwargs))
            os.makedirs(path(*args, **kwargs))
        return path(*args, **kwargs)
    return check_existence_or_create_it


def _is_input_path(path):
    """
    입력 경로 검색 함수의 출력에 적용되는 함수를 그룹화하기 위한 데코레이터 함수입니다.
    """
    @_norm_path
    @_assure_path_exists
    def check_existence(*args, **kwargs):
        return path(*args, **kwargs)
    return check_existence

@_is_input_path
def get_train_path():
    path = "./input/train/train"
    return path

@_is_input_path
def get_test_path():
    path = "./input/test/test"
    return path

@_is_input_path
def get_train_audio_path():
    path = os.path.join(get_train_path(), "audio")
    return path

@_is_input_path
def get_scoring_audio_path():
    path = os.path.join(get_test_path(), "audio")
    return path

@_is_output_path
def get_submissions_path():
    path = "../working/output"
    return path

@_is_output_path
def get_silence_path():
    path = "../working/silence"
    return path

## Utilities

일반적인 범용 유틸리티입니다.


In [ ]:
# Utilities
flatten = lambda l: [item for sublist in l for item in sublist]

def batching(iterable, n=1):
    l = len(iterable)
    for ndx in range(0, l, n):
        yield iterable[ndx:min(ndx + n, l)]

## Data Tools

Data handling tools

In [ ]:
# Data tools
def read_wav(filepath, pad=True):
    """
    wav 파일의 파일 경로를 지정하면 이 함수는 파일을 읽고 정규화하고 패드를 채웁니다.
    16k개의 샘플을 가지고 있는지 확인합니다.
    1. filepath: wav 파일의 기존 파일 경로입니다. (str)
    2. pad: 패딩이 필요합니까? (bool)
    3. returns: 표본과 목표 변수 (tuple of (np.array, str))
    """
    sample_rate, x = wavfile.read(filepath)
    target = os.path.split(os.path.split(filepath)[0])[1]
    assert sample_rate==16000
    if pad:
        return np.pad(x, (0, 16000-len(x)), mode="constant")/32768, target
    else:
        return x/32768, target

def get_batcher(list_of_paths, batch_size, label_encoder=None, scoring=False):
    """
    배치 목록이 지정된 배치 생성기를 작성합니다.
    1. list_of_paths: 형식 요소가 있는 튜플 리스트입니다. (filepath, target) (list)
    2. batch_size: size of the batch (int)
    3. label_encoder: fitted LabelEncoder (sklearn.LabelEncoder|optional)
    4. scoring: 대상을 고려해야 합니까? (bool)
    5. returns: batch generator
    """
    for filepaths in batching(list_of_paths, batch_size):
        wavs, targets = zip(*list(map(read_wav, filepaths)))
        if scoring:
            yield np.expand_dims(np.row_stack(wavs), 2), filepaths
        else:
            if label_encoder is None:
                yield np.expand_dims(np.row_stack(wavs), 2), np.row_stack(targets)
            else:
                yield np.expand_dims(np.row_stack(wavs), 2), np.expand_dims(label_encoder.transform(np.squeeze(targets)),1)


## Architecture building blocks

Inception-1D(일명 wavception)는 이 문제를 해결하기 위해 몇 주 전에 설계한 모듈입니다. 그것은 규칙적인 컨볼루션 뉴럴 네트의 성능을 상당히 향상시킵니다.

In [ ]:
class BatchNorm(object):
    def __init__(self, epsilon=1e-5, momentum=0.999, name="batch_norm"):
        with tf.variable_scope(name):
            self.epsilon = epsilon
            self.momentum = momentum
            self.name = name

    def __call__(self, x, train=True):
        return tf.contrib.layers.batch_norm(x,
                                            decay=self.momentum,
                                            updates_collections=None,
                                            epsilon=self.epsilon,
                                            scale=True,
                                            is_training=train,
                                            scope=self.name)
    
def inception_1d(x, is_train, depth, norm_function, activ_function, name):  # 
    """
    (x=self.placeholders.wav_in, is_train=self.placeholders.is_train, 
                             norm_function=BatchNorm, activ_function=tf.nn.relu, depth=1,
                             name="Inception_1_1")
                             
    Inception 1D 모듈 구현입니다.
    1. x: 현재 모듈에 입력합니다. (4D tensor with channels-last)
    2. is_train: BatchNormalization 동작을 제어하기 위한 부울 자리 표시자가 되려고 합니다. (0D tensor)
    3. depth: 네트워크의 깊이를 선형적으로 제어합니다.(int)
    4. norm_function: 정규화 클래스입니다. (same format as the BatchNorm class above)
    5. activ_function: 활성화함수 (e.g. tf.nn.relu) 
    6. name: 변수 범위의 이름입니다. (str)
    """
    with tf.variable_scope(name):
        x_norm = norm_function(name="norm_input")(x, train=is_train) # BatchNorm(name='norm_input')(x, train=s)

        # Branch 1: 64 x conv 1x1 
        branch_conv_1_1 = tf.layers.conv1d(inputs=x_norm, filters=16*depth, kernel_size=1,
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                           padding="same", name="conv_1_1")
        branch_conv_1_1 = norm_function(name="norm_conv_1_1")(branch_conv_1_1, train=is_train) # BatchNorm
        branch_conv_1_1 = activ_function(branch_conv_1_1, "activation_1_1")  # tf.nn.relu

        # Branch 2: 128 x conv 3x3 
        branch_conv_3_3 = tf.layers.conv1d(inputs=x_norm, filters=16, kernel_size=1, 
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                           padding="same", name="conv_3_3_1")
        branch_conv_3_3 = norm_function(name="norm_conv_3_3_1")(branch_conv_3_3, train=is_train)
        branch_conv_3_3 = activ_function(branch_conv_3_3, "activation_3_3_1")

        branch_conv_3_3 = tf.layers.conv1d(inputs=branch_conv_3_3, filters=32*depth, kernel_size=3, 
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                           padding="same", name="conv_3_3_2")
        branch_conv_3_3 = norm_function(name="norm_conv_3_3_2")(branch_conv_3_3, train=is_train)
        branch_conv_3_3 = activ_function(branch_conv_3_3, "activation_3_3_2")

        # Branch 3: 128 x conv 5x5 
        branch_conv_5_5 = tf.layers.conv1d(inputs=x_norm, filters=16, kernel_size=1, 
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                           padding="same", name="conv_5_5_1")
        branch_conv_5_5 = norm_function(name="norm_conv_5_5_1")(branch_conv_5_5, train=is_train)
        branch_conv_5_5 = activ_function(branch_conv_5_5, "activation_5_5_1")

        branch_conv_5_5 = tf.layers.conv1d(inputs=branch_conv_5_5, filters=32*depth, kernel_size=5, 
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                           padding="same", name="conv_5_5_2")
        branch_conv_5_5 = norm_function(name="norm_conv_5_5_2")(branch_conv_5_5, train=is_train)
        branch_conv_5_5 = activ_function(branch_conv_5_5, "activation_5_5_2")

        # Branch 4: 128 x conv 7x7
        branch_conv_7_7 = tf.layers.conv1d(inputs=x_norm, filters=16, kernel_size=1, 
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                           padding="same", name="conv_7_7_1")
        branch_conv_7_7 = norm_function(name="norm_conv_7_7_1")(branch_conv_7_7, train=is_train)
        branch_conv_7_7 = activ_function(branch_conv_7_7, "activation_7_7_1")

        branch_conv_7_7 = tf.layers.conv1d(inputs=branch_conv_7_7, filters=32*depth, kernel_size=5, 
                                           kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                           padding="same", name="conv_7_7_2")
        branch_conv_7_7 = norm_function(name="norm_conv_7_7_2")(branch_conv_7_7, train=is_train)
        branch_conv_7_7 = activ_function(branch_conv_7_7, "activation_7_7_2")

        # Branch 5: 16 x (max_pool 3x3 + conv 1x1)
        branch_maxpool_3_3 = tf.layers.max_pooling1d(inputs=x_norm, pool_size=3, strides=1, padding="same", name="maxpool_3")
        branch_maxpool_3_3 = norm_function(name="norm_maxpool_3_3")(branch_maxpool_3_3, train=is_train)
        branch_maxpool_3_3 = tf.layers.conv1d(inputs=branch_maxpool_3_3, filters=16, kernel_size=1, 
                                              kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                              padding="same", name="conv_maxpool_3")

        # Branch 6: 16 x (max_pool 5x5 + conv 1x1)
        branch_maxpool_5_5 = tf.layers.max_pooling1d(inputs=x_norm, pool_size=5, strides=1, padding="same", name="maxpool_5")
        branch_maxpool_5_5 = norm_function(name="norm_maxpool_5_5")(branch_maxpool_5_5, train=is_train)
        branch_maxpool_5_5 = tf.layers.conv1d(inputs=branch_maxpool_5_5, filters=16, kernel_size=1, 
                                              kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                              padding="same", name="conv_maxpool_5")

        # Branch 7: 16 x (avg_pool 3x3 + conv 1x1)
        branch_avgpool_3_3 = tf.layers.average_pooling1d(inputs=x_norm, pool_size=3, strides=1, padding="same", name="avgpool_3")
        branch_avgpool_3_3 = norm_function(name="norm_avgpool_3_3")(branch_avgpool_3_3, train=is_train)
        branch_avgpool_3_3 = tf.layers.conv1d(inputs=branch_avgpool_3_3, filters=16, kernel_size=1,
                                              kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                              padding="same", name="conv_avgpool_3")

        # Branch 8: 16 x (avg_pool 5x5 + conv 1x1)
        branch_avgpool_5_5 = tf.layers.average_pooling1d(inputs=x_norm, pool_size=5, strides=1, padding="same", name="avgpool_5")
        branch_avgpool_5_5 = norm_function(name="norm_avgpool_5_5")(branch_avgpool_5_5, train=is_train)
        branch_avgpool_5_5 = tf.layers.conv1d(inputs=branch_avgpool_5_5, filters=16, kernel_size=1, 
                                              kernel_initializer=tf.contrib.layers.xavier_initializer(),
                                              padding="same", name="conv_avgpool_5")

        # Concatenate
        output = tf.concat([branch_conv_1_1, branch_conv_3_3, branch_conv_5_5, branch_conv_7_7, branch_maxpool_3_3, 
                           branch_maxpool_5_5, branch_avgpool_3_3, branch_avgpool_5_5], axis=-1)
        return output
